# Bonus Unit 1: Observability and Evaluation of Agents

In this tutorial, **we're going to monitor the steps of your agent and evaluate it**

This notebook is part of the <a href="https://www.hf.co/learn/agents-course/unit1/introduction">Hugging Face Agents Course</a>, a free Course from beginner to expert, where you learn to build Agents.

<img src="https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png" alt="Agent Course"/>





## Exercise Prerequisites 🏗️

Before diving into the notebook, you need to:

🔲 📚 **Study [Introduction to Agents](https://huggingface.co/learn/agents-course/unit1/introduction) Section**

🔲 📚 **Study [The smolagents framework](https://huggingface.co/learn/agents-course/unit2/smolagents/introduction) Section**

## Step 0: Install the required libraries

In [ ]:
%pip install 'smolagents[telemetry]'
%pip install opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents
%pip install langfuse datasets 'smolagents[gradio]' gradio

## Step 1: Instrument your agent

In this notebook we will be using Langfuse as observability tool but you can use any other tool.

In this notebook, we'll be using the [smolagents framework](https://huggingface.co/learn/agents-course/unit2/smolagents/introduction). 

If you want to learn how to instriment other agent frameworks, please refer to the docs:
- LangGraph
- LlamaIndex

In [1]:
import os
import base64
 
LANGFUSE_PUBLIC_KEY = "pk-lf-..."
LANGFUSE_SECRET_KEY = "sk-lf-..."
LANGFUSE_AUTH=base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()
 
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel" # EU data region
# os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # US data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"
 
# your Hugging Face token
os.environ["HF_TOKEN"] = "hf_..."

os.environ["SERPAPI_API_KEY"] = "XXX"

In [2]:
# Get keys for your project from the project settings page
# https://cloud.langfuse.com

os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." # DOCS EXAMPLE KEYS
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." # DOCS EXAMPLE KEYS
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

from langfuse import Langfuse
langfuse = Langfuse()


In [3]:
from opentelemetry.sdk.trace import TracerProvider
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
 
trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

# Sets the global default tracer provider
from opentelemetry import trace
trace.set_tracer_provider(trace_provider)
# Creates a tracer from the global tracer provider
tracer = trace.get_tracer(__name__)
 
SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

/Users/jannik/Documents/GitHub/hf-agents-course/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 2: Test your Instrumentation

Run a simple somlagent to test your instrumentation.

In [8]:
from smolagents import HfApiModel, CodeAgent

agent = CodeAgent(tools=[], model=HfApiModel(), additional_authorized_imports=['requests', 'bs4'])
agent.run("Could you get me the title of the page at url 'https://huggingface.co/blog'?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could you get me the title of the page at url 'https://huggingface.co/blog'?                                    │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import requests                                                                                                  
  from bs4 import BeautifulSoup                                                                                    
                                                                                                                   
  response = requests.get(url='https://huggingface.co/blog')                                                       
  soup = BeautifulSoup(response.text, 'html.parser')                                                               
  title = soup.find('title').text                                                                                  
  print(title)                                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Hugging Face – Blog

Out: None

[Step 1: Duration 1.38 seconds| Input tokens: 2,036 | Output tokens: 98]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import requests                                                                                                  
  from bs4 import BeautifulSoup                                                                                    
                                                                                                                   
  response = requests.get(url='https://huggingface.co/blog')                                                       
  soup = BeautifulSoup(response.text, 'html.parser')                                                               
  title = soup.find('title').text                                                                                  
  print(title)                                                                                                     
  final_answer(answer=title)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Hugging Face – Blog

Out - Final answer: Hugging Face – Blog

[Step 2: Duration 1.16 seconds| Input tokens: 4,291 | Output tokens: 198]

'Hugging Face – Blog'

Check that the traces are being sent to Langfuse by going to the [Traces](https://cloud.langfuse.com/traces) page.

![Example trace in Langfuse](https://langfuse.com/images/cookbook/huggingface-agent-course/first-example-trace.png)

_[Link to the trace](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/489efdfbe80923ce9869edde0aaaf61a?timestamp=2025-03-07T10%3A27%3A35.606Z)_

## Metrics

Now we use a more complex agent, to see what kind of metrics we can get.

In [55]:
from smolagents import (CodeAgent, DuckDuckGoSearchTool, GoogleSearchTool, HfApiModel)

search_tool = GoogleSearchTool()
 
agent = CodeAgent(tools=[search_tool], model=HfApiModel())

agent.run("How many Rubik's Cubes could you fit inside the Notre Dame Cathedral?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How many Rubik's Cubes could you fit inside the Notre Dame Cathedral?                                           │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  cube_dimensions_query = "standard rubiks cube dimensions in cm"                                                  
  cathedral_dimensions_query = "interior dimensions of notre dame cathedral in meters"                             
                                                                                                                   
  cube_dimensions = web_search(query=cube_dimensions_query)                                                        
  cathedral_dimensions = web_search(query=cathedral_dimensions_query)                                              
                                                                                                                   
  print("Cube dimensions:", cube_dimensions)                                                                       
  print("Cathedral dimensions:", cathedral_dimensions)                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Cube dimensions: ## Search Results
0. [Rubik's Cube](https://en.wikipedia.org/wiki/Rubik%27s_Cube)
Source: Wikipedia

A standard Rubik's Cube measures 5.6 centimetres (2+1⁄4 in) on each side. The puzzle consists of 26 unique 
miniature cubes, also known as "cubies" or "cubelets ...

1. [The shrinking cube trend, what's the perfect size? : 
r/Cubers](https://www.reddit.com/r/Cubers/comments/908k5s/the_shrinking_cube_trend_whats_the_perfect_size/)
Source: Reddit · r/Cubers

For me a 3x3 should be somewhere between 5 and 6 cm. Can be a little more or a little less but not too much.

2. [Rubik's Cube](https://www.firstversions.com/2015/08/rubiks-cube.html)
Source: First Versions

Features: A standard Rubik's Cube measures 5.7 cm (approximately 2¼ inches) on each side. The puzzle consists of 
twenty-six unique miniature cubes, also called ...

3. [What are the standard dimensions for a rubik's 
cube?](https://toys.games.narkive.com/SWtyt3m9/what-are-the-standard-dimensions-for-a-rubik-s-cube)
Source: narkive

Rubiks Cube Dimensions. 11 years ago. I measured my cube and it is exactly 5.7 cm in length, width, and height. Any
other cubes that are smaller than that are ...

4. [Rubik's Cube](https://www.target.com/p/rubik-39-s-cube/-/A-14783169)
Source: Target

Q: what is the dimensions of the cube ... A: It's most likely 5.7 cm (or 2.25 inch) per side. That's the standard 
Rubik's cube size. submitted by ...

5. [Rubik'S Cube 
(5.7Cm)](https://iwantgift.com/products/rubiks-cube-5-7cm?srsltid=AfmBOorMEkhrIjIbtDce6vtNonXaCTJlaXuc8xqAAKWr-A5TQ
-0Ppl6M)
Source: I Want Gift

Dimensions: 5.7x5.7x5.7cm. Prices quoted are for standard product printing position and size as shown below:.

6. [MindCuber can solve standard size Rubik's 
Cube...](https://www.facebook.com/lego.mindcuber/posts/mindcuber-can-solve-standard-size-rubiks-cube-puzzles-about-
57cm-across-but-his-/464406060289309/)
Source: Facebook · MindCuber

MindCuber can solve standard size Rubik's Cube puzzles about 5.7cm across, but his `bot friends can solve many 
others.

7. [The Official Rubik's Cube | Make Your Move](https://www.rubiks.com/)
Source: Rubik's Cube

There is only one Rubik's Cube and it changed the world. Make your Move today and start your Rubik's Cube journey! 
Find everything you need to know about ...
Cathedral dimensions: ## Search Results
0. [The Majestic Layout of 
Notre-Dame](https://www.friendsofnotredamedeparis.org/notre-dame-cathedral/architecture/layout/)
Source: Friends of Notre-Dame de Paris

Total area: over 6,000 m2 (64,000 ft2) · Length: 128 m (420 ft) · Width: 48 m (157 ft) · Tower height: 69 m (226 
ft) · Façade width: 43.5 m (142 ft) · Façade height ...

1. [Plans and Elevations](https://www.notredamedeparis.fr/en/understand/architecture/plans/)
Source: Notre-Dame de Paris

Overall area: over 6,000 m² · Length: 127 m · Width: 48 m · Height of the towers: 69 m · Width of the façade: 43.5 
m · Height of the façade without the towers: 45 m ...

2. [Notre-Dame Cathedral Paris | History, Facts, 
Style](https://www.friendsofnotredamedeparis.org/notre-dame-cathedral/)
Source: Friends of Notre-Dame de Paris

Length: 420 feet (128 meters) · Width: 157 feet (48 meters) · Nave roof height: 115 feet (35 meters) · Tower 
height: 226 feet (69 meters) · Spire height: 315 feet ( ...

3. [Notre-Dame de Paris | History, Style, Fire, & Facts](https://www.britannica.com/topic/Notre-Dame-de-Paris)
Date published: Jan 31, 2025
Source: Britannica

The interior of the cathedral is 427 by 157 feet (130 by 48 meters) in plan, and the roof is 115 feet (35 meters) 
high. Two massive early ...

4. [Some facts and figures about Notre Dame Cathedral](https://apnews.com/article/1ddf4e7d38db44cb814f3672123db2a9)
Date published: Apr 15, 2019
Source: AP News

The interior of the cathedral is 427 feet by 157 feet (130 meters by 48 meters). The main structure is 211 feet 
high (64 meters high).

5. [Interior view by ARCHITECT, French](https://www.wga.hu/html_m/zzzarc

[Step 1: Duration 13.61 seconds| Input tokens: 2,108 | Output tokens: 174]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Rubik's Cube dimensions in cm                                                                                  
  cube_side_length_cm = 5.7                                                                                        
                                                                                                                   
  # Notre Dame Cathedral dimensions in meters, converting to cm                                                    
  cathedral_length_cm = 130 * 100                                                                                  
  cathedral_width_cm = 48 * 100                                                                                    
  cathedral_height_cm = 35 * 100                                                                                   
                                                                                                                   
  # Volume of the Rubik's Cube in cm^3                                                                             
  cube_volume_cm3 = cube_side_length_cm ** 3                                                                       
                                                                                                                   
  # Volume of the Notre Dame Cathedral in cm^3                                                                     
  cathedral_volume_cm3 = cathedral_length_cm * cathedral_width_cm * cathedral_height_cm                            
                                                                                                                   
  # Number of Rubik's Cubes that can fit inside the cathedral                                                      
  num_cubes = cathedral_volume_cm3 // cube_volume_cm3                                                              
                                                                                                                   
  final_answer(int(num_cubes))                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 1179310233

[Step 2: Duration 16.75 seconds| Input tokens: 6,077 | Output tokens: 466]

1179310233

### Trace Structure

In most observability tools, the trace is the main unit of observation. It contains all the spans of the agent's run. For this example run, one trace has been created. This trace contains spans for each step of the agent's run.

![Trace](https://langfuse.com/images/cookbook/huggingface-agent-course/trace-tree.png)

_[Link to the trace](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/1ac33b89ffd5e75d4265b62900c348ed?timestamp=2025-03-07T13%3A45%3A09.149Z&display=preview)_

## Online Evaluation

As we discussed in the previous section, there are two ways to make sense of observability data:
- Online Evaluation
- Offline Evaluation

Online Evaluation refers to a live monitoring of an agent that is running in production. For this, teams usually capture the following metrics: 

### Costs

As the the instrumentation also captures the token usage of the agent, this can be shown as well. We can also calculate the costs of the model usage by setting the price per token.

![Costs](https://langfuse.com/images/cookbook/huggingface-agent-course/smolagents-costs.png)

_[Link to the trace](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/1ac33b89ffd5e75d4265b62900c348ed?timestamp=2025-03-07T13%3A45%3A09.149Z&display=preview)_

### Latency

By tracing the steps of the agent, you can see the latency of each step. In this example run, we can see that the agent took around 32 seconds to complete the task.

![Latency](https://langfuse.com/images/cookbook/huggingface-agent-course/smolagents-latency.png)
_[Link to the trace](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/1ac33b89ffd5e75d4265b62900c348ed?timestamp=2025-03-07T13%3A45%3A09.149Z&display=preview)_

### Other metrics and application logic

When running your agent in production, it is also important to capture additional metrics of your agent run such as a user id, session id, or other metadata. As smolagents use OpenTelemetry, it is possible to pass additional attributes to the spans. When using Langfuse, this can be done by passing the `langfuse.user.id`, `langfuse.session.id` and `langfuse.tags` attributes.

In [30]:
from smolagents import (CodeAgent, GoogleSearchTool, HfApiModel)

search_tool = GoogleSearchTool()
agent = CodeAgent(tools=[search_tool], model=HfApiModel(model_id="https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud/v1/chat/completions"))

with tracer.start_as_current_span("Smolagent-Trace") as span:
    span.set_attribute("langfuse.user.id", "smolagent-user-123")
    span.set_attribute("langfuse.session.id", "smolagent-session-123456789")
    span.set_attribute("langfuse.tags", ["city-question", "testing-agents"])

    agent.run("What is the capital of Germany?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the capital of Germany?                                                                                 │
│                                                                                                                 │
╰─ HfApiModel - https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud/v1/chat/completions ───────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = web_search(query="capital of Germany")                                                                  
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [Capital of Germany](https://en.wikipedia.org/wiki/Capital_of_Germany)
Source: Wikipedia

The capital of Germany is the City Land of Berlin. It is the seat of the Federal President of Germany, whose 
official residence is Bellevue Castle.

1. [Berlin](https://en.wikipedia.org/wiki/Berlin)
Source: Wikipedia

Berlin is the capital and largest city of Germany, by both area and population. With 3.66 million inhabitants, it 
has the highest population within its city ...

2. [Berlin - Official Website of the City of Berlin, Capital of ...](https://www.berlin.de/en/)
Source: Berlin.de

Berlin - Official Website of the City of Berlin, Capital of Germany – Berlin.de.

3. [Berlin | History, Map, Population, Attractions, & Facts](https://www.britannica.com/place/Berlin)
Source: Britannica

Berlin is the capital and chief urban center of Germany. Berlin was the capital of Prussia and then, from 1871, of 
a unified Germany. Though partitioned into ...

4. [How did Berlin Become the Capital of Reunified 
Germany?](https://www.britannica.com/video/Overview-Bonn-decision-Berlin-Germany/-197060)
Source: Britannica

Overview of the decision to make Berlin—rather than Bonn—the capital of reunified Germany.

5. [Germany | Facts, Geography, Maps, & History](https://www.britannica.com/place/Germany)
Source: Britannica

Head Of Government: Chancellor: Olaf Scholz ; Capital: Berlin ; Population: (2025 est.) 83,433,000.

6. [State Capitals of Germany](https://www.deutschland.de/en/topic/life/mobility-travel/state-capitals)
Date published: Jun 20, 2013
Source: deutschland.de

State Capitals of Germany · Berlin · Bremen · Dresden · Düsseldorf · Erfurt · Hamburg · Hannover · Kiel.

7. [Berlin The new and fascinating capital of 
Germany](https://amuraworld.com/en/topics/by-the-world/articles/788-berlin-the-new-and-fascinating-capital-of-germa
ny)
Source: Amura Yachts & Lifestyle

Berlin is a world that fills one with history as you walk through its streets, its palaces and y plazas and avenues
dotted with romantic buildings.

8. [CAPITAL OF GERMANY 🇩🇪 it is ♥️ BERLIN here we come ...](https://www.youtube.com/watch?v=f4UK_nCsIho)
Source: YouTube · Jayesh & Shreya

Here we are in the capital of Germany. Its been long time we wanted to visit Berlin and finally we have made it. 
Stay tuned for our Berlin ...

Out: None

[Step 1: Duration 2.91 seconds| Input tokens: 2,099 | Output tokens: 50]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Berlin")                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Berlin

[Step 2: Duration 1.88 seconds| Input tokens: 4,937 | Output tokens: 92]

![Enhancing agent runs with additional metrics](https://langfuse.com/images/cookbook/huggingface-agent-course/smolagents-attributes.png)

These metrics can then be analyzed over time to see how the agent's performance evolves. Observability tools give you an overview of these metrics to spot regressions or improvements early on.

![Observability metrics overview in Langfuse](https://langfuse.com/images/cookbook/huggingface-agent-course/langfuse-dashboard.png)

### User feedback

If the agent is embedded in a UI, the user feedback can be captured as well. In this example, the user clicked on a "tumps up" button which can be captured in an observability tool.

In [ ]:
from smolagents import (
    load_tool,
    CodeAgent,
    HfApiModel,
    GradioUI
)

# Import tool from Hub
image_generation_tool = load_tool("m-ric/text-to-image", trust_remote_code=True)

model = HfApiModel()

# Initialize the agent with the image generation tool
agent = CodeAgent(tools=[image_generation_tool], model=model)

GradioUI(agent).launch()

### Implicit user feedback

### LLM-as-a-Judge

LLM-as-a-Judge is a technique to evaluate the quality of the agent's output by using a LLM to judge the quality of the output. This is a common technique to evaluate the quality of the agent's output over time. 

For this, you first need to create an Evaluation Template. 
Then 

## Offline Evaluation

Equally important to monitoring your agent live in production is to run systematic evaluations during development or before updating your agent. This is also refered to as offline evaluation.

### Dataset Evaluation

To evaluate your agent, before putting it in front of users, you usually test your agent against datasets that represent the tasks the agent is supposed to perform. For this you create a dataset that contains user input queries and the expected output. These inputs are then passed to the agent and the output is compared to the expected output.

In this example, we will use the [GSM8K dataset](https://huggingface.co/datasets/gsm8k) from Hugging Face to test the math solving capabilities of the agent. First, we fetch the dataset form Hugging Face into a pandas DataFrame.

In [55]:
import pandas as pd
from datasets import load_dataset

# 1. Fetch the GSM8K dataset from Huggingface into a pandas DataFrame.
# Here, we load the training split.
dataset = load_dataset("openai/gsm8k", 'main', split='train')
df = pd.DataFrame(dataset)
print("First few rows of GSM8K dataset:")
print(df.head())

First few rows of GSM8K dataset:
                                            question  \
0  Natalia sold clips to 48 of her friends in Apr...   
1  Weng earns $12 an hour for babysitting. Yester...   
2  Betty is saving money for a new wallet which c...   
3  Julie is reading a 120-page book. Yesterday, s...   
4  James writes a 3-page letter to 2 different fr...   

                                              answer  
0  Natalia sold 48/2 = <<48/2=24>>24 clips in May...  
1  Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...  
2  In the beginning, Betty has only 100 / 2 = $<<...  
3  Maila read 12 x 2 = <<12*2=24>>24 pages today....  
4  He writes each friend 3*2=<<3*2=6>>6 pages a w...  


Then, we create a Langfuse dataset and add the dataset items. This setp helps us to track the performance on different dataset runs. However, you can also directly loop over the pandas dataframe if you are using another observability solution.

In [56]:
from langfuse import Langfuse

langfuse = Langfuse()

# 2. Create a Langfuse dataset using the langfuse client.
langfuse_dataset_name = "gsm8k_dataset"

langfuse.create_dataset(
    name=langfuse_dataset_name,
    description="GSM8K benchmark dataset uploaded from Huggingface",
    metadata={
        "date": "2025-03-07",  # Adjust the date as needed
        "type": "benchmark"
    }
)
print("Langfuse dataset created successfully.")

Langfuse dataset created successfully.


In [57]:
# 3. Upload dataset items from the DataFrame to Langfuse.
# For each row, we create a dataset item where:
# - The "input" contains the question text.
# - The "expected_output" contains the answer text.
# Optionally, we include metadata (e.g. the original row index).
# Here we upload the first 10 rows; adjust as needed.

for idx, row in df.iterrows():
    langfuse.create_dataset_item(
        dataset_name=langfuse_dataset_name,
        input={"text": row["question"]},
        expected_output={"text": row["answer"]},
        metadata={"source_index": idx}
    )
    if idx >= 9:  # Upload only first 50 items for this example.
        break

print("Uploaded dataset items to Langfuse.")

Uploaded dataset items to Langfuse.


Next, we define the function that will be used to run the agent. Here we can also pass additional attributes to the span. We are also fetching the OpenTelemetry trace id to link the trace to the dataset item.


In [58]:
from opentelemetry.trace import format_span_id, format_trace_id
from smolagents import (
    CodeAgent,
    HfApiModel,
)

model = HfApiModel(model_id="https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud/v1/chat/completions")
agent = CodeAgent(tools=[], model=model, add_base_tools=True)

def run_smolagent(question):

    with tracer.start_as_current_span("Smolagent-Trace") as span:
        span.set_attribute("langfuse.tag", "dataset-run")

        output = agent.run(question)

        # Get the current span
        current_span = trace.get_current_span()
        span_context = current_span.get_span_context()
        trace_id = span_context.trace_id
        formatted_trace_id = format_trace_id(trace_id)
        print(formatted_trace_id)

        langfuse_trace = langfuse.trace(id=formatted_trace_id, input=question, output=output)
        
    return langfuse_trace, output

In this step, we loop over the dataset items and execute the agent. We also link the trace to the dataset item and give it a run_name.

In [ ]:
from langfuse import Langfuse

langfuse = Langfuse()

dataset = langfuse.get_dataset(langfuse_dataset_name)
 
for item in dataset.items:
    # execute application function and get langfuse_object (trace/span/generation/event)
    # output also returned as it is used to evaluate the run
    # you can also link using ids, see sdk reference for details

    langfuse_trace, output = run_smolagent(item.input["text"])
 
    # link the execution trace to the dataset item and give it a run_name
    item.link(
        langfuse_trace,
        run_name="smolagent-notebook-run-05",
        run_description="My first run", # optional
        run_metadata={ "model": model } # optional
    )
 
    # optionally, evaluate the output to compare different runs more easily
    langfuse_trace.score(
        name="<example_eval>",
        # any float value
        value=1.5,
        comment="This is a comment" # optional, useful to add reasoning
    )
 
# Flush the langfuse client to ensure all data is sent to the server at the end of the experiment run
langfuse.flush()


We can then repeat this process with different Language models, search tools or system prompts and compare how the agent performs.